In [1]:
import os
os.chdir(r"C:\Users\KutayTaha\Desktop\diş_eti")
print(os.getcwd())


c:\Users\KutayTaha\Desktop\diş_eti


# ilk

In [2]:
import os
from PIL import Image
from collections import defaultdict


ModuleNotFoundError: No module named 'PIL'

In [3]:
BASE_DIR = "Dataset"   # proje kökünden Dataset klasörü
SPLITS = ["training", "validation", "test"]


In [4]:
def check_dataset(split):
    img_dir = os.path.join(BASE_DIR, split, "images")
    lbl_dir = os.path.join(BASE_DIR, split, "labels")

    images = {os.path.splitext(f)[0] for f in os.listdir(img_dir)}
    labels = {os.path.splitext(f)[0] for f in os.listdir(lbl_dir)}

    missing_labels = images - labels      # resmi var label yok
    missing_images = labels - images      # label var resim yok

    corrupted_images = []
    for img in os.listdir(img_dir):
        img_path = os.path.join(img_dir, img)
        try:
            Image.open(img_path).verify()
        except Exception:
            corrupted_images.append(img)

    return missing_labels, missing_images, corrupted_images


In [5]:
results = {}

for split in SPLITS:
    results[split] = check_dataset(split)

results



{'training': (set(), set(), []),
 'validation': (set(), set(), []),
 'test': (set(), set(), [])}

In [6]:
def check_label_format(label_path):
    errors = []

    with open(label_path, "r") as f:
        for i, line in enumerate(f):
            parts = line.strip().split()

            if len(parts) != 5:
                errors.append(f"Satır {i}: 5 eleman yok")
                continue

            try:
                cls = int(parts[0])
                coords = list(map(float, parts[1:]))

                if not all(0 <= c <= 1 for c in coords):
                    errors.append(f"Satır {i}: Koordinatlar 0-1 dışında")
            except:
                errors.append(f"Satır {i}: Sayısal dönüşüm hatası")

    return errors



In [7]:
label_errors = defaultdict(list)

for split in SPLITS:
    lbl_dir = os.path.join(BASE_DIR, split, "labels")

    for lbl in os.listdir(lbl_dir):
        path = os.path.join(lbl_dir, lbl)
        errs = check_label_format(path)

        if errs:
            label_errors[split].append((lbl, errs))

label_errors


defaultdict(list, {'training': [('00598.txt', ['Satır 14: 5 eleman yok'])]})

In [10]:
label_path = "Dataset/training/labels/00598.txt"

with open(label_path, "r") as f:
    for i, line in enumerate(f, start=1):
        if i == 14:
            print(f"Satır {i}: '{line.strip()}'")



Satır 14: '0 0.541304 0.520257 0.584239 0.289699'


In [11]:
def clean_all_labels():
    for split in ["training", "validation", "test"]:
        lbl_dir = f"Dataset/{split}/labels"

        for lbl in os.listdir(lbl_dir):
            path = os.path.join(lbl_dir, lbl)

            with open(path, "r") as f:
                lines = f.readlines()

            cleaned = [
                line for line in lines
                if len(line.strip().split()) == 5
            ]

            if len(cleaned) != len(lines):
                with open(path, "w") as f:
                    f.writelines(cleaned)
                print(f"Düzeltildi: {split}/{lbl}")

clean_all_labels()


Düzeltildi: training/00598.txt


In [12]:
from collections import Counter

class_counter = Counter()

for split in SPLITS:
    lbl_dir = os.path.join(BASE_DIR, split, "labels")

    for lbl in os.listdir(lbl_dir):
        with open(os.path.join(lbl_dir, lbl)) as f:
            for line in f:
                cls = int(line.split()[0])
                class_counter[cls] += 1

class_counter


Counter({5: 4867, 4: 3851, 6: 2292, 3: 1831, 0: 1096, 1: 1096, 2: 309})

# class inbalance düzeltme

In [16]:
import os
import shutil
import random
from collections import Counter

LABEL_DIR = "Dataset/Training/labels"
IMAGE_DIR = "Dataset/Training/images"

TARGET_COUNT = 1000  # az sınıflar için hedef

# sınıf sayıları
class_counts = Counter()

label_files = os.listdir(LABEL_DIR)

for lf in label_files:
    with open(os.path.join(LABEL_DIR, lf)) as f:
        for line in f:
            cls = int(line.split()[0])
            class_counts[cls] += 1

print("Mevcut dağılım:", class_counts)

# az olan sınıflar
minor_classes = [c for c, n in class_counts.items() if n < TARGET_COUNT]

print("Çoğaltılacak sınıflar:", minor_classes)

for cls in minor_classes:
    needed = TARGET_COUNT - class_counts[cls]

    cls_files = []
    for lf in label_files:
        with open(os.path.join(LABEL_DIR, lf)) as f:
            for line in f:
                if int(line.split()[0]) == cls:
                    cls_files.append(lf)
                    break

    for i in range(needed):
        src_label = random.choice(cls_files)
        src_img = src_label.replace(".txt", ".jpg")

        new_label = f"aug_{cls}_{i}_{src_label}"
        new_img = new_label.replace(".txt", ".jpg")

        shutil.copy(
            os.path.join(LABEL_DIR, src_label),
            os.path.join(LABEL_DIR, new_label)
        )
        shutil.copy(
            os.path.join(IMAGE_DIR, src_img),
            os.path.join(IMAGE_DIR, new_img)
        )

print("Oversampling tamamlandı.")


Mevcut dağılım: Counter({5: 3399, 4: 2178, 6: 1751, 3: 1218, 0: 732, 1: 732, 2: 244})
Çoğaltılacak sınıflar: [0, 1, 2]
Oversampling tamamlandı.


# eğitim

In [4]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")

model.train(
    data="dataset.yaml",
    epochs=30,
    imgsz=640,
    batch=8,
    name="dis_eti_final2"
)

New https://pypi.org/project/ultralytics/8.4.9 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.230 🚀 Python-3.11.9 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=dataset.yaml, epochs=30, time=None, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=dis_eti_final27, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, 

train: Scanning C:\Users\KutayTaha\Desktop\diş_eti\Dataset\training\labels.cache... 2024 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2024/2024 [00:00<?, ?it/s]

train: C:\Users\KutayTaha\Desktop\diş_eti\Dataset\training\images\00060.jpg: 1 duplicate labels removed
train: C:\Users\KutayTaha\Desktop\diş_eti\Dataset\training\images\00085.jpg: 1 duplicate labels removed
train: C:\Users\KutayTaha\Desktop\diş_eti\Dataset\training\images\00484.jpg: 1 duplicate labels removed
train: C:\Users\KutayTaha\Desktop\diş_eti\Dataset\training\images\aug_0_179_00085.jpg: 1 duplicate labels removed
train: C:\Users\KutayTaha\Desktop\diş_eti\Dataset\training\images\aug_2_122_00085.jpg: 1 duplicate labels removed
train: C:\Users\KutayTaha\Desktop\diş_eti\Dataset\training\images\aug_2_137_00060.jpg: 1 duplicate labels removed
train: C:\Users\KutayTaha\Desktop\diş_eti\Dataset\training\images\aug_2_194_00060.jpg: 1 duplicate labels removed
train: C:\Users\KutayTaha\Desktop\diş_eti\Dataset\training\images\aug_2_227_00060.jpg: 1 duplicate labels removed
train: C:\Users\KutayTaha\Desktop\diş_eti\Dataset\training\images\aug_2_275_00060.jpg: 1 duplicate labels removed
trai


val: Scanning C:\Users\KutayTaha\Desktop\diş_eti\Dataset\validation\labels.cache... 182 images, 0 backgrounds, 0 corrupt: 100%|██████████| 182/182 [00:00<?, ?it/s]

val: C:\Users\KutayTaha\Desktop\diş_eti\Dataset\validation\images\00811.jpg: 1 duplicate labels removed


...

                   all        182       2548      0.524      0.655      0.515      0.361



30 epochs completed in 1.073 hours.
Optimizer stripped from runs\detect\dis_eti_final27\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\dis_eti_final27\weights\best.pt, 22.5MB

Validating runs\detect\dis_eti_final27\weights\best.pt...
Ultralytics YOLOv8.0.230 🚀 Python-3.11.9 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
Model summary (fused): 168 layers, 11128293 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:09<00:00,  1.33it/s]


                   all        182       2548       0.49       0.69      0.544      0.374
              saglikli        182        182      0.996          1      0.995      0.848
      hafif_gingivitis        182        182      0.995          1      0.995      0.784
      ileri_gingivitis        182         30     0.0981      0.433      0.178      0.108
         periodontitis        182        306       0.21      0.209      0.199      0.109
                  plak        182        774      0.482      0.545      0.478      0.259
                tartar        182        754       0.38      0.799      0.422       0.23
                kanama        182        320      0.273      0.844      0.538      0.281
Speed: 0.6ms preprocess, 4.1ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to runs\detect\dis_eti_final27


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000205AF706550>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047

# Predict

In [7]:
import os

for root, dirs, files in os.walk("runs"):
    for file in files:
        if file == "best.pt":
            print(os.path.join(root, file))


runs\detect\dis_eti_final27\weights\best.pt
runs\detect\runs\dis_eti_yolo_gpu\weights\best.pt


In [8]:
import shutil
import os

src = r"runs\detect\dis_eti_final27\weights\best.pt"
dst_dir = r"C:\Users\KutayTaha\Desktop\diş_eti\models"

os.makedirs(dst_dir, exist_ok=True)
shutil.copy(src, dst_dir)

print("✅ Model başarıyla kaydedildi:", dst_dir)


✅ Model başarıyla kaydedildi: C:\Users\KutayTaha\Desktop\diş_eti\models


In [15]:
from ultralytics import YOLO

model = YOLO("C:/Users/KutayTaha/Desktop/diş_eti/models/best.pt")

model.predict(
    source="Dataset/test/images/00929.jpg",
    conf=0.5,   # 0.25 yerine 0.5
    save=True
)





image 1/1 C:\Users\KutayTaha\Desktop\diş_eti\Dataset\test\images\00929.jpg: 448x640 1 saglikli, 1 hafif_gingivitis, 42.9ms
Speed: 4.0ms preprocess, 42.9ms inference, 29.9ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs\detect\predict2


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'saglikli', 1: 'hafif_gingivitis', 2: 'ileri_gingivitis', 3: 'periodontitis', 4: 'plak', 5: 'tartar', 6: 'kanama'}
 orig_img: array([[[52, 64, 92],
         [54, 66, 94],
         [56, 68, 96],
         ...,
         [48, 61, 93],
         [42, 55, 87],
         [36, 49, 81]],
 
        [[50, 62, 90],
         [50, 62, 90],
         [54, 66, 94],
         ...,
         [47, 60, 92],
         [45, 58, 90],
         [40, 53, 85]],
 
        [[53, 65, 93],
         [50, 62, 90],
         [54, 66, 94],
         ...,
         [45, 58, 90],
         [47, 60, 92],
         [47, 60, 92]],
 
        ...,
 
        [[25, 36, 56],
         [24, 35, 55],
         [27, 36, 56],
         ...,
         [42, 54, 82],
         [40, 52, 80],
         [37, 49, 77]],
 
        [[27, 38, 58],
         [25, 36, 56],
         [27, 36, 56],
         ...,
     

In [16]:
results = model.predict("Dataset/test/images/00929.jpg", conf=0.5)

img = results[0].plot(
    labels=True,
    conf=False,     # score kapalı
    line_width=3    # daha kalın kutu
)

import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.imshow(img)
plt.axis("off")
plt.show()



image 1/1 C:\Users\KutayTaha\Desktop\diş_eti\Dataset\test\images\00929.jpg: 448x640 1 saglikli, 1 hafif_gingivitis, 170.5ms
Speed: 8.0ms preprocess, 170.5ms inference, 4.0ms postprocess per image at shape (1, 3, 448, 640)


<Figure size 1000x600 with 1 Axes>

# modeli tekrar kullanmak için 

In [ ]:
from ultralytics import YOLO

model = YOLO("C:/Users/KutayTaha/Desktop/diş_eti/models/best.pt")



FileNotFoundError: [Errno 2] No such file or directory: 'Desktop\\diş_eti\\models\\best.pt'

In [3]:

model.predict(
    source="Dataset/test/images/00929.jpg",
    conf=0.5,   # 0.25 yerine 0.5
    save=True
)



image 1/1 c:\Users\KutayTaha\Desktop\diş_eti\Dataset\test\images\00929.jpg: 448x640 1 saglikli, 1 hafif_gingivitis, 77.8ms
Speed: 7.0ms preprocess, 77.8ms inference, 130.7ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs\detect\predict3


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'saglikli', 1: 'hafif_gingivitis', 2: 'ileri_gingivitis', 3: 'periodontitis', 4: 'plak', 5: 'tartar', 6: 'kanama'}
 orig_img: array([[[52, 64, 92],
         [54, 66, 94],
         [56, 68, 96],
         ...,
         [48, 61, 93],
         [42, 55, 87],
         [36, 49, 81]],
 
        [[50, 62, 90],
         [50, 62, 90],
         [54, 66, 94],
         ...,
         [47, 60, 92],
         [45, 58, 90],
         [40, 53, 85]],
 
        [[53, 65, 93],
         [50, 62, 90],
         [54, 66, 94],
         ...,
         [45, 58, 90],
         [47, 60, 92],
         [47, 60, 92]],
 
        ...,
 
        [[25, 36, 56],
         [24, 35, 55],
         [27, 36, 56],
         ...,
         [42, 54, 82],
         [40, 52, 80],
         [37, 49, 77]],
 
        [[27, 38, 58],
         [25, 36, 56],
         [27, 36, 56],
         ...,
     

# model kontrolü 



yolo task=detect mode=val model="C:\Users\KutayTaha\Desktop\diş_eti\runs\detect\dis_eti_final27\weights\best.pt" data=dataset.yaml


                   all        182       2548       0.49      0.688      0.544      0.374
              saglikli        182        182      0.996          1      0.995      0.847
      hafif_gingivitis        182        182      0.995          1      0.995      0.784
      ileri_gingivitis        182         30     0.0988      0.433      0.177      0.108
         periodontitis        182        306      0.205      0.202      0.199      0.108
                  plak        182        774      0.482       0.54      0.479       0.26
                tartar        182        754      0.382      0.798      0.423       0.23
                kanama        182        320      0.274      0.844      0.538      0.283
                tartar        182        754      0.382      0.798      0.423       0.23
                tartar        182        754      0.382      0.798      0.423       0.23
                kanama        182        320      0.274      0.844      0.538      0.283

### ÇOK İYİ OLANLAR

saglikli
hafif_gingivitis


### ORTA – KABUL EDİLEBİLİR

plak
tartar
kanama


### ZAYIF OLANLAR

ileri_gingivitis
periodontitis


## Özellikle zayıf sınıflar:

ileri_gingivitis → mAP50: 0.177

periodontitis → mAP50: 0.199

tartar → mAP50: 0.423

Ama şuna dikkat:

saglikli ve hafif_gingivitis neredeyse kusursuz

Demek ki model öğrenebiliyor, sorun:

sınıf dengesizliği

zor sınıflar (görsel olarak karışık)

Yani bu kötü model değil, zor veri problemi.